In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 41 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 10.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=43f7472ac8a874f7e7f6f7937476b723c4b631e206f676bbeda214ad0fb5999d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 12.2 MB/s 
     |████████████████████████████████| 769 kB 12.9 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 3.0 MB 52.2 MB/s 


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-zd0xogvy
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-zd0xogvy
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=4e850500d2d851995a11a72665caec4d5734ce235219ff8ec96960c9432ee113
  Stored in directory: /tmp/pip-ephem-wheel-cache-5685764_/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
import pandas as pd
title_data = pd.read_excel('/content/title_test.xlsx')

In [62]:
title_data.sample(n=10)

,News_num,document,label
905,905,"[단독]강희석 이마트 대표, 이베이 직원들에 ""온-오프 완성형 에코시스템 구축하자""",A
411,411,"[서초동 법썰] ""대놓고 음란행위"" vs ""노상방뇨일 뿐""",C
795,795,"30대 여성 살해유기 혐의 60대 男 검거…경찰 ""시신 수색중""",A
704,704,"유엔난민기구 ""연말까지 아프간 난민 50만 명...3천500억 원 필요""",A
368,368,"""다른 언론개혁법과 함께 처리해야""…커지는 신중론",C
505,505,"美 연준 의장 ""올해 안에 테이퍼링 시작 가능""",A
0,0,"""지역 환경산업 건전한 육성·발전 앞장""",C
956,956,"""백신접종 확대 중요...접종률 70% 아이슬란드 최근 사망자 없어""",C
643,643,"김용호, 유튜브 중단…""술에 무너져, 윤희숙처럼 내려놓겠다""",C
829,829,"13명 추가 도착…""정부 직접 고용한 사람들이 구출 대상""",A


In [63]:
title_data.loc[(title_data['label'] == "A"), 'label'] = 0  
title_data.loc[(title_data['label'] == "B"), 'label'] = 1  
title_data.loc[(title_data['label'] == "C"), 'label'] = 2  


In [64]:
title_data.head(5)

,News_num,document,label
0,0,"""지역 환경산업 건전한 육성·발전 앞장""",2
1,1,"""마스크 쓰고, 비누로 손 씻고""…인간 따라하는 침팬지[영상]",2
2,2,"""얀센 백신 추가 접종하면 항체 9배 높아져""",2
3,3,"민주노총, 현대제철에서 대규모 집회 강행...""고발 등 행정조치""",1
4,4,"골드만삭스 ""9월 7일부터 백신 접종자만 건물 출입"" [강영연의 뉴욕오프닝]",2


In [65]:
title_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966 entries, 0 to 965
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   News_num  966 non-null    int64 
 1   document  966 non-null    object
 2   label     966 non-null    object
dtypes: int64(1), object(2)
memory usage: 22.8+ KB


In [66]:
title_list = []
for title, label in zip(title_data['document'], title_data['label'])  :
    data = []
    data.append(title)
    data.append(str(label))

    title_list.append(data)

In [67]:
title_list[0]

['"지역 환경산업 건전한 육성·발전 앞장"', '2']

In [68]:
from sklearn.model_selection import train_test_split
                                                         
title_train, title_test = train_test_split(title_list, test_size=0.2, random_state=0)

In [69]:
print(len(title_train))
print(len(title_test))
title_train[0]

772
194


['"미 정보당국, \'코로나19 기원=미궁\' 바이든에 보고"', '2']

In [70]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [71]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [72]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train = BERTDataset(title_train, 0, 1, tok, max_len, True, False)
test = BERTDataset(title_test, 0, 1, tok, max_len, True, False)

using cached model


In [73]:
train[0]

(array([   2,  518, 2149, 4103, 5805,  517,   46,  520, 4665, 6079, 5655,
         116, 1258, 7020,  631, 2149, 5521,  520, 2186, 7096, 5928, 6896,
        2358,  518,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(25, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [74]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [75]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [76]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [77]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.2507721185684204 train acc 0.203125
epoch 1 train acc 0.41947115384615385


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.53125


  0%|          | 0/13 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.8301456570625305 train acc 0.671875
epoch 5 train acc 0.6790865384615384


  0%|          | 0/4 [00:00<?, ?it/s]

epoch 5 test acc 0.53515625


In [150]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
          
            logits=i
            logits = np.round(logits.detach().cpu().numpy(),2)

            print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("판단유보")
            elif np.argmax(logits) == 1:
                test_eval.append("책임회피")
            elif np.argmax(logits) == 2:
                test_eval.append("선정주의")
        print(">> 이타이틀은 판단유보 {:.2f}, 책임회피 {:.2f}, 선정주의 {:.2f} 으로 측정되어 {}유형의 따옴표입니다".format(logits[0],logits[1],logits[2],test_eval[0]))
        
        

using cached model


In [151]:
sen = '"디지털·교통인프라 협력 확대"…韓·콜롬비아 정상회담'
predict(sen)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[ 0.3   0.06 -0.31]
>> 이타이틀은 판단유보 0.30, 책임회피 0.06, 선정주의 -0.31 으로 측정되어 판단유보유형의 따옴표입니다


In [132]:
test_data = pd.read_excel('/content/test.xlsx')

In [133]:
test_list = []
for title in test_data['title'] :
    data = []
    data.append(title)

    test_list.append(data)

In [143]:
for i in range(len(test_list)):
  print(test_list[i][0])
  predict(test_list[i][0])
  print('\n')

"사망 유치원생을 귀신으로?"… '심야괴담회' 방송에 유족 반발


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[-0.54 -0.47  1.68]
>> 이타이틀은 판단유보 -0.54, 책임회피 -0.47, 선정주의 1.68 으로 측정되어 선정주의유형의 따옴표입니다


박영진 "마포에 신혼집 사자는 아내 말 안 들어, 지금 가격이…"(컬투쇼)
[-0.51 -0.48  1.67]
>> 이타이틀은 판단유보 -0.51, 책임회피 -0.48, 선정주의 1.67 으로 측정되어 선정주의유형의 따옴표입니다


이성경 "한석규, 연기 배우지 않았으면 좋겠다고 조언해"
[-0.08 -0.21  0.93]
>> 이타이틀은 판단유보 -0.08, 책임회피 -0.21, 선정주의 0.93 으로 측정되어 선정주의유형의 따옴표입니다


최영애 인권위원장 "한국 협력 아프간인들 국내 이송 환영"
[ 0.52 -0.09 -0.91]
>> 이타이틀은 판단유보 0.52, 책임회피 -0.09, 선정주의 -0.91 으로 측정되어 판단유보유형의 따옴표입니다


고승범 "가계부채 필요시 추가 규제…실수요자는 배려"(종합2보)
[0.05 0.04 0.47]
>> 이타이틀은 판단유보 0.05, 책임회피 0.04, 선정주의 0.47 으로 측정되어 선정주의유형의 따옴표입니다


[e글중심] "2년 당긴 고교학점제, 괜찮을까요?"
[-0.48 -0.49  1.64]
>> 이타이틀은 판단유보 -0.48, 책임회피 -0.49, 선정주의 1.64 으로 측정되어 선정주의유형의 따옴표입니다


고승범 "대출 연장·이자 유예 등 소상공인 고충 감안해야"
[ 0.53 -0.08 -0.97]
>> 이타이틀은 판단유보 0.53, 책임회피 -0.08, 선정주의 -0.97 으로 측정되어 판단유보유형의 따옴표입니다


사모펀드 CEO 후속제재 영향 불가피…금융권"예견된 결과"
[ 0.51 -0.07 -0.97]
>> 이타이틀은 판단유보 0.51, 책임회피 -0.07, 선정주의 -0.97 으로 측정되어 판단유보유형의 따옴표입니다


文대통령이 軍노마스크 지시?…국방부 "심각한 명예훼손"
[ 0.5  -0.06 -0.9 ]
>> 이타이틀은 판단유

In [137]:
for i in range(len(test_list)) :
  predict(test_list[i][0])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[-0.54 -0.47  1.68]
>> 이타이틀은 판단유보 -0.54, 책임회피 -0.47, 선정주의 1.68 으로 측정되어 선정주의유형의 따옴표입니다
[-0.51 -0.48  1.67]
>> 이타이틀은 판단유보 -0.51, 책임회피 -0.48, 선정주의 1.67 으로 측정되어 선정주의유형의 따옴표입니다
[-0.08 -0.21  0.93]
>> 이타이틀은 판단유보 -0.08, 책임회피 -0.21, 선정주의 0.93 으로 측정되어 선정주의유형의 따옴표입니다
[ 0.52 -0.09 -0.91]
>> 이타이틀은 판단유보 0.52, 책임회피 -0.09, 선정주의 -0.91 으로 측정되어 판단유보유형의 따옴표입니다
[0.05 0.04 0.47]
>> 이타이틀은 판단유보 0.05, 책임회피 0.04, 선정주의 0.47 으로 측정되어 선정주의유형의 따옴표입니다
[-0.48 -0.49  1.64]
>> 이타이틀은 판단유보 -0.48, 책임회피 -0.49, 선정주의 1.64 으로 측정되어 선정주의유형의 따옴표입니다
[ 0.53 -0.08 -0.97]
>> 이타이틀은 판단유보 0.53, 책임회피 -0.08, 선정주의 -0.97 으로 측정되어 판단유보유형의 따옴표입니다
[ 0.51 -0.07 -0.97]
>> 이타이틀은 판단유보 0.51, 책임회피 -0.07, 선정주의 -0.97 으로 측정되어 판단유보유형의 따옴표입니다
[ 0.5  -0.06 -0.9 ]
>> 이타이틀은 판단유보 0.50, 책임회피 -0.06, 선정주의 -0.90 으로 측정되어 판단유보유형의 따옴표입니다
[-0.35 -0.46  1.48]
>> 이타이틀은 판단유보 -0.35, 책임회피 -0.46, 선정주의 1.48 으로 측정되어 선정주의유형의 따옴표입니다


In [49]:
test_bert = TESTDataset(test_list, 0, tok, max_len, True, False)